In [9]:
import random
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model
from math import ceil

In [10]:
def highlight_diff(data, other, color='#ff616b'):
    attr = f'background-color: {color}'
    return pd.DataFrame(np.where(data.ne(other), attr, ''),
                        index=data.index, columns=data.columns)

def highlight_cells(val):
    color = '#1eff00' if val == 1 else ''
    return 'color: %s' % color


In [11]:

class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, num_staff, num_days, l, preferences, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._num_staff = num_staff
        self._num_days = num_days
        self._leave = l
        self._solution_count = 0
        self._solution_limit = limit
        self._preference_matrix = preferences


    def on_solution_callback(self):

        self._solution_count += 1

        print(f"Solution {self._solution_count}")

    

        solutionArray = [[self.value(self._leave[(s, d)]) for d in range(self._num_days)] for s in range(self._num_staff)]

        df = pd.DataFrame(solutionArray, columns=[f'Day {d+1}' for d in range(self._num_days)], index=[f'Employee {x+1}' for x in range(self._num_staff)])

        df2 = pd.DataFrame(self._preference_matrix, columns=[f'Day {d+1}' for d in range(self._num_days)], index=[f'Employee {x+1}' for x in range(self._num_staff)])
        
        df_styled = df.style.applymap(highlight_cells).apply(highlight_diff, axis=None, other=df2)

        display(df_styled)


        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.stop_search()



    def solutionCount(self):
        return self._solution_count

In [12]:
class Problem:

    daily_quotas = None
    staff_leave_allowance = None
    preference_matrix = None
    model = None
    solver = None
    solution = None


    # Constructor for the Problem class
    # num_staff: number of staff members
    # num_days: number of days in the problem
    # For quota, can choose between defining an array or giving limits for data to be randomly generated
    # quotaLimits: tuple of min and max quota limits (optional)
    # quotaArray: list of quotas for each day (optional)
    # For leave allowance, can choose between defining an array or giving limits for data to be randomly generated
    # leaveAllowanceLimits: tuple of min and max leave allowance limits (optional)
    # leaveAllowanceArray: list of leave allowances for each staff member (optional)
    # for preference, can choose between defining a matrix or giving percentage of requests for data to be randomly generated
    # preferencePercentage: percentage of preference for each staff member (optional)
    # preferenceMatrix: list of preferences for each staff member (optional)
    def __init__(self, num_staff: int, num_days: int, quotaLimits: tuple = None, quotaArray: list = None, leaveAllowanceLimits: tuple = None, leaveAllowanceArray: list = None, preferencePercentage:int = None, preferenceMatrix: list = None):

        # get number of staff and number of days
        self.num_staff = num_staff
        self.num_days = num_days            

        # get daily quotas, either randomly generate or get from array
        if quotaLimits:
            self.daily_quotas = [((random.randrange(quotaLimits[0], quotaLimits[1]) / 100)) for i in range(self.num_days)] 

        elif quotaArray:
            if not len(quotaArray) == self.num_days:
                raise ValueError("Quota array must have the same length as the number of days in the problem")
            
            self.daily_quotas = quotaArray

        # get staff leave allowance, either randomly generate or get from array
        if leaveAllowanceLimits:
            self.staff_leave_allowance = [random.randint(leaveAllowanceLimits[0], leaveAllowanceLimits[1]) for i in range(self.num_staff)]

        elif leaveAllowanceArray:
            if not len(leaveAllowanceArray) == self.num_staff:
                raise ValueError("Leave allowance array must have the same length as the number of staff in the problem")
            
            self.staff_leave_allowance = leaveAllowanceArray

        # get preference matrix, either randomly generate or get from array
        if not preferenceMatrix:
            self.generate_preference_matrix(preferencePercentage)
        else:    
            self.preference_matrix = np.array(preferenceMatrix).reshape(self.num_staff, self.num_days)

        # check that all values are defined
        if not self.daily_quotas:
            raise ValueError("Quotas must be defined")
        
        if not self.staff_leave_allowance:
            raise ValueError("Staff leave allowance must be defined")
        
        if self.preference_matrix.any() == None:
            raise ValueError("Preference matrix must be defined")
        
    def __str__(self):
        return f"{self.num_staff} staff, {self.num_days} days,\nquotas:\n{self.daily_quotas},\nstaff leave allowance:\n{self.staff_leave_allowance}\npreference matrix:\n{self.preference_matrix}"
        

    def generate_preference_matrix(self, preferencePercentage):
        num_ones = int(self.num_staff * self.num_days * (preferencePercentage / 100))
        num_zeros = (self.num_staff * self.num_days) - num_ones
        array = np.array([1] * num_ones + [0] * num_zeros)
        np.random.shuffle(array)
        self.preference_matrix = array.reshape(self.num_staff,self.num_days)

    def dataset_to_csv(self, filename):
        ## ADD OTHER METRICS TO FILE
        df = pd.DataFrame(self.preference_matrix)
        df.to_csv(filename, index=False, header=False)

    # def from_csv(self, filename):
        # df = pd.read_csv(filename, header=None)

    def createModel(self):
        self.model = cp_model.CpModel()

        # Matrix for leave granted
        # if employee e has day d off, then L[e,d] = 1
        self.l = {}
        for e in range(self.num_staff):
            for d in range(self.num_days):
                self.l[(e, d)] = self.model.new_bool_var(f"L_{e}_d{d}")

        # Staff leave allowance not exceeded
        for e in range(self.num_staff):
            self.model.Add(sum(self.l[e, d] for d in range(self.num_days)) <= self.staff_leave_allowance[e])
        
        # daily leave quota not exceeded
        # calculate number of staff * staff_limit for each day
        for d in range(self.num_days):
            self.model.Add(sum(self.l[e, d] for e in range(self.num_staff)) <= ceil(self.num_staff * self.daily_quotas[d]))

        #############################################
        # Objective function:
        # maximise preferences satisfied
        objective_1 = sum(self.preference_matrix[e][d] * self.l[e, d] for e in range(self.num_staff) for d in range(self.num_days))

        objective_1_weighting = 5

        # maximise number of consecutive days off
        consecutive_days_off = {}

        for e in range(self.num_staff):
            for d in range(self.num_days-1):
                consecutive_days_off[(e, d)] = self.model.new_bool_var(f"consecutive_days_off_{e}_{d}")

        for e in range(self.num_staff):
            for d in range(self.num_days-1):
                self.model.add_multiplication_equality(consecutive_days_off[(e, d)], [self.l[e, d], self.l[e, d+1]])

        objective_2 = sum(consecutive_days_off[(e, d)] for e in range(self.num_staff) for d in range(self.num_days-1))

        objective_2_weighting = 4


        # give preference to days off with higher leave allowance
        objective_3 = sum(self.staff_leave_allowance[e] * self.l[e, d] for e in range(self.num_staff) for d in range(self.num_days))
        objective_3_weighting = 2

        # Objective function
        self.model.maximize(
                (objective_1_weighting * objective_1) + 
                (objective_2_weighting * objective_2) +
                (objective_3_weighting * objective_3)
            )   

        pass

    # Solve the model
    def solve(self):
        self.solver = cp_model.CpSolver()
        self.solver.parameters.enumerate_all_solutions = True
        self.solution = self.solver.Solve(self.model)

    # get the results of the model
    def getResults(self):
        if self.solution == cp_model.OPTIMAL or self.solution == cp_model.FEASIBLE:

            solutionArray = [[self.solver.Value(self.l[s,d]) for d in range(self.num_days)] for s in range(self.num_staff)]

            df = pd.DataFrame(solutionArray, columns=[f'Day {d+1}' for d in range(self.num_days)], index=[f'Employee {x+1}' for x in range(self.num_staff)])

            df2 = pd.DataFrame(self.preference_matrix, columns=[f'Day {d+1}' for d in range(self.num_days)], index=[f'Employee {x+1}' for x in range(self.num_staff)])


            df_styled = df.style.applymap(highlight_cells).apply(highlight_diff, axis=None, other=df2)

            return df_styled

        else:
            print("No feasible solution found.")

    def generateSolution(self):
        self.createModel()
        self.solve()

    def list_all_solutions(self):

        solution_limit = 10
        solution_printer = SolutionPrinter(self.num_staff, self.num_days, self.l, self.preference_matrix, solution_limit)
        solver = cp_model.CpSolver()
        solver.parameters.enumerate_all_solutions = True

        solver.Solve(self.model, solution_printer)
        





In [13]:
problem = Problem(5, 10, quotaLimits=(0, 25), leaveAllowanceLimits=(0, 3), preferencePercentage=50)
print(problem)
problem.generateSolution()
problem.getResults()
problem.list_all_solutions()

5 staff, 10 days,
quotas:
[0.23, 0.15, 0.13, 0.13, 0.08, 0.02, 0.0, 0.01, 0.08, 0.0],
staff leave allowance:
[3, 0, 3, 3, 1]
preference matrix:
[[1 0 1 1 0 0 0 1 0 0]
 [0 0 1 0 0 1 1 1 0 1]
 [0 1 0 1 1 0 1 1 1 1]
 [0 1 1 0 0 0 0 0 1 1]
 [1 1 0 1 0 1 1 0 0 0]]
Solution 1


C:\Users\seren\AppData\Local\Temp\ipykernel_30888\3168802148.py:156: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_styled = df.style.applymap(highlight_cells).apply(highlight_diff, axis=None, other=df2)
C:\Users\seren\AppData\Local\Temp\ipykernel_30888\1867692167.py:27: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_styled = df.style.applymap(highlight_cells).apply(highlight_diff, axis=None, other=df2)


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
Employee 1,1,0,1,0,0,0,0,0,0,0
Employee 2,0,0,0,0,0,0,0,0,0,0
Employee 3,0,0,0,1,1,1,0,0,0,0
Employee 4,0,1,0,0,0,0,0,1,1,0
Employee 5,1,0,0,0,0,0,0,0,0,0


Solution 2


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
Employee 1,1,0,1,1,0,0,0,0,0,0
Employee 2,0,0,0,0,0,0,0,0,0,0
Employee 3,0,0,0,0,1,1,0,1,0,0
Employee 4,1,1,0,0,0,0,0,0,1,0
Employee 5,0,0,0,0,0,0,0,0,0,0


Solution 3


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
Employee 1,1,0,0,1,0,0,0,0,0,0
Employee 2,0,0,0,0,0,0,0,0,0,0
Employee 3,0,0,0,0,1,0,0,1,1,0
Employee 4,1,1,1,0,0,0,0,0,0,0
Employee 5,0,0,0,0,0,1,0,0,0,0


Solution 4


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
Employee 1,1,0,0,0,0,0,0,1,1,0
Employee 2,0,0,0,0,0,0,0,0,0,0
Employee 3,0,0,0,1,1,1,0,0,0,0
Employee 4,1,1,1,0,0,0,0,0,0,0
Employee 5,0,0,0,0,0,0,0,0,0,0
